rown_number(), rank, dense_rank, lead, lag

In [0]:
sampleData = [
('Tom Prescott', 'Furniture', 'Chairs', 4000),
('Quincy Jones', 'Furniture', 'Bookcases', 4000),
('Joseph Holt', 'Furniture', 'Tables', 5000),
('Alejandro Grove', 'Furniture', 'Furnishings', 8000),
('Adrian Barton', 'Office Supplies', 'Binders', 3000),
('Ken Lonsdale', 'Office Supplies', 'Supplies', 9000),
('Greg Guthrie', 'Office Supplies', 'Fasteners', 3000),
('Yoseph Carroll', 'Office Supplies', 'Storage', 3000),
('Sean Miller', 'Technology', 'Machines', 22000),
('Tamara Chand', 'Technology', 'Copiers', 18000),
('John Murray', 'Technology', 'Phones', 5000),
('Kelly Collister', 'Technology', 'Accessories', 3000)
]

In [0]:
# column names for dataframe
columns = ["Customer_Name","Category","Subcategory","Sales"]
# create dataframe
df = spark.createDataFrame(data = sampleData,schema = columns)

In [0]:
df.show()

+---------------+---------------+-----------+-----+
|  Customer_Name|       Category|Subcategory|Sales|
+---------------+---------------+-----------+-----+
|   Tom Prescott|      Furniture|     Chairs| 4000|
|   Quincy Jones|      Furniture|  Bookcases| 4000|
|    Joseph Holt|      Furniture|     Tables| 5000|
|Alejandro Grove|      Furniture|Furnishings| 8000|
|  Adrian Barton|Office Supplies|    Binders| 3000|
|   Ken Lonsdale|Office Supplies|   Supplies| 9000|
|   Greg Guthrie|Office Supplies|  Fasteners| 3000|
| Yoseph Carroll|Office Supplies|    Storage| 3000|
|    Sean Miller|     Technology|   Machines|22000|
|   Tamara Chand|     Technology|    Copiers|18000|
|    John Murray|     Technology|     Phones| 5000|
|Kelly Collister|     Technology|Accessories| 3000|
+---------------+---------------+-----------+-----+



In [0]:
from pyspark.sql.functions import * # this has to used whenever you are working with windows functions
from pyspark.sql.window import *   # this has to used whenever you are trying to use transformation funcrtions
from pyspark.sql.types import *    # this has to used whenever you are trying to assign datatypes and strcut  field and struct type
from pyspark.sql.column import *   # this has to used whenever you are  working with col in dataframe

* to create a window function in pyspark, there are two steps 
   * step1 : assign a partition and oreder level, either table or specific column partition levels
   * teps2 : write any fuynction based on above partion created

Example for SQL tabel level : over (order by sales dec)
pyspark code for tabel level: Window.orderBy(col("Sales").desc())

Example for SQL partition level: over (partition by category order by sales desc)
pyspark code for partition level: Window.partitionBy("Category").orderBy(col("Sales").desc()

In [0]:
#create a row_number() for above table based on sales descending order
# step:1: create a windows partition at table level
table_partition = Window.orderBy(col("Sales").desc())

#step2: create a row_number function

df2 = df.withColumn("row_number",row_number().over(table_partition))
df2.show()


/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+---------------+-----------+-----+----------+
|  Customer_Name|       Category|Subcategory|Sales|row_number|
+---------------+---------------+-----------+-----+----------+
|    Sean Miller|     Technology|   Machines|22000|         1|
|   Tamara Chand|     Technology|    Copiers|18000|         2|
|   Ken Lonsdale|Office Supplies|   Supplies| 9000|         3|
|Alejandro Grove|      Furniture|Furnishings| 8000|         4|
|    Joseph Holt|      Furniture|     Tables| 5000|         5|
|    John Murray|     Technology|     Phones| 5000|         6|
|   Tom Prescott|      Furniture|     Chairs| 4000|         7|
|   Quincy Jones|      Furniture|  Bookcases| 4000|         8|
|  Adrian Barton|Office Supplies|    Binders| 3000|         9|
|   Greg Guthrie|Office Supplies|  Fasteners| 3000|        10|
| Yoseph Carroll|Office Supplies|    Storage| 3000|        11|
|Kelly Collister|     Technology|Accessories| 3000|        12|
+---------------+---------------+-----------+-----+----

In [0]:
# craete a row_number() for above table based on category partiton and sales descending

# Step1: create a windows partition at at level
window_cat = Window.partitionBy("Category").orderBy(col("Sales").desc())

#step2:  Create a row_number function
df = df.withColumn("row_number",row_number().over(window_cat))\
.withColumn("Rank",rank().over(window_cat))\
    .withColumn("Dense_rank",dense_rank().over(window_cat))
df.show()

+---------------+---------------+-----------+-----+----------+----+----------+
|  Customer_Name|       Category|Subcategory|Sales|row_number|Rank|Dense_rank|
+---------------+---------------+-----------+-----+----------+----+----------+
|Alejandro Grove|      Furniture|Furnishings| 8000|         1|   1|         1|
|    Joseph Holt|      Furniture|     Tables| 5000|         2|   2|         2|
|   Tom Prescott|      Furniture|     Chairs| 4000|         3|   3|         3|
|   Quincy Jones|      Furniture|  Bookcases| 4000|         4|   3|         3|
|   Ken Lonsdale|Office Supplies|   Supplies| 9000|         1|   1|         1|
|  Adrian Barton|Office Supplies|    Binders| 3000|         2|   2|         2|
|   Greg Guthrie|Office Supplies|  Fasteners| 3000|         3|   2|         2|
| Yoseph Carroll|Office Supplies|    Storage| 3000|         4|   2|         2|
|    Sean Miller|     Technology|   Machines|22000|         1|   1|         1|
|   Tamara Chand|     Technology|    Copiers|18000| 

*LED : Get the next row value into current row
** syntax: lead("COLNAME",<NO_OF_NextColumn>).over(<PARTITION LEVEL>)
* LAG : get egh last row value into currrent row
** syntax: lag("COLNAME",<no_of_lastcolumn>).over(<PARTITION LEVEL>)

In [0]:
# get teh previous row sales into current rtow
#step1: Create a window partiotn at table level
table_partition = Window.orderBy(col("Sales").desc())
#step2: Create a lag function
d2f = df.withColumn("lag1",lag("Sales",1).over(table_partition))\
    .withColumn("lag2",lag("Sales",2).over(table_partition))
d2f.show()

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+---------------+-----------+-----+----------+----+----------+-----+-----+
|  Customer_Name|       Category|Subcategory|Sales|row_number|Rank|Dense_rank| lag1| lag2|
+---------------+---------------+-----------+-----+----------+----+----------+-----+-----+
|    Sean Miller|     Technology|   Machines|22000|         1|   1|         1| NULL| NULL|
|   Tamara Chand|     Technology|    Copiers|18000|         2|   2|         2|22000| NULL|
|   Ken Lonsdale|Office Supplies|   Supplies| 9000|         1|   1|         1|18000|22000|
|Alejandro Grove|      Furniture|Furnishings| 8000|         1|   1|         1| 9000|18000|
|    Joseph Holt|      Furniture|     Tables| 5000|         2|   2|         2| 8000| 9000|
|    John Murray|     Technology|     Phones| 5000|         3|   3|         3| 5000| 8000|
|   Tom Prescott|      Furniture|     Chairs| 4000|         3|   3|         3| 5000| 5000|
|   Quincy Jones|      Furniture|  Bookcases| 4000|         4|   3|         3| 4000| 5000|

In [0]:
#get the next row sales into current row

#step1: creta a window partiotn at table level
table_partition = Window.orderBy(col("Sales").desc())
#step2: Create a lead function
df2 = df.withColumn("lead1",lead("Sales",1).over(table_partition))\
    .withColumn("lead2",lead("Sales",2).over(table_partition))
df2.show()

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------------+---------------+-----------+-----+----------+----+----------+-----+-----+
|  Customer_Name|       Category|Subcategory|Sales|row_number|Rank|Dense_rank|lead1|lead2|
+---------------+---------------+-----------+-----+----------+----+----------+-----+-----+
|    Sean Miller|     Technology|   Machines|22000|         1|   1|         1|18000| 9000|
|   Tamara Chand|     Technology|    Copiers|18000|         2|   2|         2| 9000| 8000|
|   Ken Lonsdale|Office Supplies|   Supplies| 9000|         1|   1|         1| 8000| 5000|
|Alejandro Grove|      Furniture|Furnishings| 8000|         1|   1|         1| 5000| 5000|
|    Joseph Holt|      Furniture|     Tables| 5000|         2|   2|         2| 5000| 4000|
|    John Murray|     Technology|     Phones| 5000|         3|   3|         3| 4000| 4000|
|   Tom Prescott|      Furniture|     Chairs| 4000|         3|   3|         3| 4000| 3000|
|   Quincy Jones|      Furniture|  Bookcases| 4000|         4|   3|         3| 3000| 3000|

In [0]:
#get top 2 sales from each category
from pyspark.sql.functions import *
from pyspark.sql.window import *

partion_cat = Window.partitionBy("Category").orderBy(col("Sales").desc())
df2 = df.withColumn("rank",rank().over(partion_cat))
df2 = df2.filter("rank <=2")
df2.show()


+---------------+---------------+-----------+-----+----------+----+----------+
|  Customer_Name|       Category|Subcategory|Sales|row_number|rank|Dense_rank|
+---------------+---------------+-----------+-----+----------+----+----------+
|Alejandro Grove|      Furniture|Furnishings| 8000|         1|   1|         1|
|    Joseph Holt|      Furniture|     Tables| 5000|         2|   2|         2|
|   Ken Lonsdale|Office Supplies|   Supplies| 9000|         1|   1|         1|
|  Adrian Barton|Office Supplies|    Binders| 3000|         2|   2|         2|
|   Greg Guthrie|Office Supplies|  Fasteners| 3000|         3|   2|         2|
| Yoseph Carroll|Office Supplies|    Storage| 3000|         4|   2|         2|
|    Sean Miller|     Technology|   Machines|22000|         1|   1|         1|
|   Tamara Chand|     Technology|    Copiers|18000|         2|   2|         2|
+---------------+---------------+-----------+-----+----------+----+----------+



In [0]:
from pyspark.sql import SparkSession

# initialize spark session
spark = SparkSession.builder.appName("RemoveDuplicates").getOrCreate()

data = [
    (1, 'Alice', '2024-01-01', 'A'), (1, 'Alice', '2024-01-02', 'B'), (1, 'Alice', '2024-01-03', 'C'), (1, 'Alice', '2024-01-04', 'D'),
    (2, 'Bob', '2024-02-01', 'E'), (2, 'Bob', '2024-02-02', 'F'), (2, 'Bob', '2024-02-03', 'G'), (2, 'Bob', '2024-02-04', 'H'),
    (3, 'Charlie', '2024-03-01', 'I'), (3, 'Charlie', '2024-03-02', 'J'), (3, 'Charlie', '2024-03-03', 'K'), (3, 'Charlie', '2024-03-04', 'L'),
    (4, 'David', '2024-04-01', 'M'), (4, 'David', '2024-04-02', 'N'), (4, 'David', '2024-04-03', 'O'), (4, 'David', '2024-04-04', 'P'),
    (5, 'Emma', '2024-05-01', 'Q'), (5, 'Emma', '2024-05-02', 'R'), (5, 'Emma', '2024-05-03', 'S'), (5, 'Emma', '2024-05-04', 'T')
]
#Create Data Frame
df = spark.createDataFrame(data, ['id', 'name', 'date', 'value'])
df.show()


+---+-------+----------+-----+
| id|   name|      date|value|
+---+-------+----------+-----+
|  1|  Alice|2024-01-01|    A|
|  1|  Alice|2024-01-02|    B|
|  1|  Alice|2024-01-03|    C|
|  1|  Alice|2024-01-04|    D|
|  2|    Bob|2024-02-01|    E|
|  2|    Bob|2024-02-02|    F|
|  2|    Bob|2024-02-03|    G|
|  2|    Bob|2024-02-04|    H|
|  3|Charlie|2024-03-01|    I|
|  3|Charlie|2024-03-02|    J|
|  3|Charlie|2024-03-03|    K|
|  3|Charlie|2024-03-04|    L|
|  4|  David|2024-04-01|    M|
|  4|  David|2024-04-02|    N|
|  4|  David|2024-04-03|    O|
|  4|  David|2024-04-04|    P|
|  5|   Emma|2024-05-01|    Q|
|  5|   Emma|2024-05-02|    R|
|  5|   Emma|2024-05-03|    S|
|  5|   Emma|2024-05-04|    T|
+---+-------+----------+-----+



In [0]:
#get the lastest date transaction for each customer

part_cust = Window.partitionBy("name").orderBy(col ("date").desc())
df2 = df.withColumn("rank",rank().over(part_cust))
df2 = df2.filter("rank =1")
df2.show()

+---+-------+----------+-----+----+
| id|   name|      date|value|rank|
+---+-------+----------+-----+----+
|  1|  Alice|2024-01-04|    D|   1|
|  2|    Bob|2024-02-04|    H|   1|
|  3|Charlie|2024-03-04|    L|   1|
|  4|  David|2024-04-04|    P|   1|
|  5|   Emma|2024-05-04|    T|   1|
+---+-------+----------+-----+----+



In [0]:
# initialize spark session
 
spark = SparkSession.builder.appName("CreateDataFrame").getOrCreate() 

list1 = [
    (10000, 'Alice', '2024-01-01', 'A'),
    (10000, 'Alice', '2024-01-02', 'B'),
    (5000, 'Bob', '2024-02-01', 'E'),
    (5000, 'Bob', '2024-02-02', 'F'),
    (5000, 'Bob', '2024-02-03', 'G'),
    (5000, 'Bob', '2024-02-04', 'H'),
    (3000, 'Charlie', '2024-03-01', 'I')
]

df = spark.createDataFrame(list1, ['id', 'name', 'date', 'value'])
df =df.select(df.id.alias("AMT"))
df.show()

+-----+
|  AMT|
+-----+
|10000|
|10000|
| 5000|
| 5000|
| 5000|
| 5000|
| 3000|
+-----+



In [0]:
parnt_rank = Window.orderBy(col("AMT").desc())
df2 = df.withColumn("rowN",row_number().over(parnt_rank)).withColumn("Rank",rank().over(parnt_rank)).withColumn("dense_rank",dense_rank().over(parnt_rank))
display(df2)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


AMT,rowN,Rank,dense_rank
10000,1,1,1
10000,2,1,1
5000,3,3,2
5000,4,3,2
5000,5,3,2
5000,6,3,2
3000,7,7,3
